## Load datasets

In [ ]:
import pandas as pd
c3f=pd.read_csv(r"C:\Users\Kaveri\Downloads\CHASE datasets\c3f.csv")
iris_c=pd.read_csv(r"C:\Users\Kaveri\Downloads\ds-researcch\iris_c.csv")

In [ ]:
from sklearn.datasets import load_digits

def load_digits_dataset():
    digits = load_digits(as_frame=True)   # returns DataFrame + Series
    X = digits.data        # features (1797 × 64)
    y = digits.target      # labels (1797,)
    return X, y

# Example usage
X_digits, y_digits = load_digits_dataset()
print(X_digits.shape, y_digits.unique())


In [ ]:
from sklearn.datasets import fetch_rcv1

def load_rcv1_dataset():
    rcv1 = fetch_rcv1()
    X = rcv1.data        # sparse TF-IDF matrix (804k × 47k)
    y = rcv1.target      # multilabel (804k × 103)
    target_names = rcv1.target_names
    return X, y, target_names

# Example usage
X_rcv1, y_rcv1, labels_rcv1 = load_rcv1_dataset()
print(X_rcv1.shape, y_rcv1.shape, labels_rcv1[:10])


In [ ]:
from sklearn.datasets import fetch_20newsgroups, fetch_20newsgroups_vectorized

def load_20news_dataset(vectorized=True):
    if vectorized:
        # Already TF-IDF features
        newsgroups = fetch_20newsgroups_vectorized(subset='all')
        X = newsgroups.data      # sparse matrix (18846 × 130k)
        y = newsgroups.target    # class labels (0–19)
        target_names = newsgroups.target_names
    else:
        # Raw text version
        newsgroups = fetch_20newsgroups(subset='all')
        X = newsgroups.data
        y = newsgroups.target
        target_names = newsgroups.target_names
    
    return X, y, target_names

# Example usage
X_20news, y_20news, labels_20news = load_20news_dataset(vectorized=True)
print(X_20news.shape, len(set(y_20news)), labels_20news[:5])


In [ ]:
#load them

In [ ]:
from sklearn.datasets import load_digits
import pandas as pd

digits = load_digits(as_frame=True)
X_digits, y_digits = digits.data, digits.target

print("DIGITS DATASET")
print("Shape:", X_digits.shape)
print("Classes:", y_digits.nunique())
print(y_digits.value_counts())

# Show first 5 rows
display(X_digits.head())


In [ ]:
from sklearn.datasets import fetch_20newsgroups_vectorized
import pandas as pd

news = fetch_20newsgroups_vectorized(subset="all")
X_20news, y_20news = news.data, news.target

print("\n20 NEWSGROUPS DATASET (preview 5 docs × 10 features)")
df_20news = pd.DataFrame.sparse.from_spmatrix(X_20news[:5, :10])  # first 5 docs, 10 features
display(df_20news)


In [ ]:
from sklearn.datasets import fetch_rcv1
import pandas as pd

rcv1 = fetch_rcv1()
X_rcv1, y_rcv1 = rcv1.data, rcv1.target

print("\nRCV1 DATASET (preview 5 docs × 10 features)")
df_rcv1 = pd.DataFrame.sparse.from_spmatrix(X_rcv1[:5, :10])  # first 5 docs, 10 features
display(df_rcv1)


## Preprocess datasets into (X, y) format

In [ ]:
import numpy as np

# Take y_rcv1 (sparse matrix, 804k × 103)
# Convert to array of topic indices: choose the *first* label per doc
y_rcv1_single = y_rcv1.argmax(axis=1).A1   # pick index of max label (first positive)

print("RCV1 single-label shape:", y_rcv1_single.shape)
print("Unique classes:", np.unique(y_rcv1_single)[:10])


In [ ]:
# Already single-label multiclass
y_20news_single = y_20news  # just rename for consistency
print("20NEWS unique classes:", len(set(y_20news_single)))


In [ ]:
# Already fine #digits
y_digits_single = y_digits

In [ ]:
# Check structure
print(c3f.head())
print(c3f.columns)

# Suppose last 3 columns are targets: Cat, Sub_Cat, Label
# (adjust names if different)
target_cols = ["Cat", "Sub_Cat", "Label"]

# Features = all except target columns
X_c3f = c3f.drop(columns=target_cols)

# Targets = separate Series for each level
y_c3f_cat = c3f["Cat"]
y_c3f_sub = c3f["Sub_Cat"]
y_c3f_label = c3f["Label"]

print("Features:", X_c3f.shape, "Cat:", y_c3f_cat.nunique(),
      "SubCat:", y_c3f_sub.nunique(), "Label:", y_c3f_label.nunique())


In [ ]:
iris_c = iris_c.drop(columns=["Unnamed: 0"])

In [ ]:
iris_wide = iris_c.pivot_table(
    index="Species",              # group by species
    columns=["Part", "Measure"],  # create feature columns
    values="Value", 
    aggfunc="mean"                # average if duplicates
).reset_index()


In [ ]:
y_iris = iris_wide["Species"]              # target
X_iris = iris_wide.drop(columns=["Species"])  # features

print(X_iris.shape, y_iris.nunique(), y_iris.unique())


In [ ]:
#encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Encode species into integers
le_iris = LabelEncoder()
y_iris_encoded = le_iris.fit_transform(y_iris)

print("Iris Classes:", le_iris.classes_)
print("Sample Encoded Labels:", np.unique(y_iris_encoded)[:10])




In [ ]:
# If Unnamed: 0 still exists, drop it safely
if "Unnamed: 0" in iris_c.columns:
    iris_c = iris_c.drop(columns=["Unnamed: 0"])

# Pivot into wide format: each row = one flower
iris_wide = iris_c.pivot_table(
    index=iris_c.index,             # keep row identity
    columns=["Part", "Measure"],    # make sepal_length, sepal_width, etc.
    values="Value",
    aggfunc="first"
)

# Add Species back as target
iris_wide["Species"] = iris_c["Species"]

# Features and target
X_iris = iris_wide.drop(columns=["Species"])
y_iris = iris_wide["Species"]

# Encode target labels
from sklearn.preprocessing import LabelEncoder
le_iris = LabelEncoder()
y_iris_encoded = le_iris.fit_transform(y_iris)

print("X shape:", X_iris.shape)
print("y length:", len(y_iris_encoded))
print("Classes:", le_iris.classes_)


In [ ]:
# Create artificial sample_id (every 4 rows = 1 sample)
iris_c = iris_c.copy()
iris_c["sample_id"] = iris_c.index // 4

# Pivot into wide format
iris_wide = iris_c.pivot_table(
    index="sample_id",
    columns=["Part", "Measure"],
    values="Value",
    aggfunc="first"
)

# Add target back (species) - take the first value per sample_id
iris_wide["Species"] = iris_c.groupby("sample_id")["Species"].first()

# Features and target
X_iris = iris_wide.drop(columns=["Species"])
y_iris = iris_wide["Species"]

# Encode
from sklearn.preprocessing import LabelEncoder
le_iris = LabelEncoder()
y_iris_encoded = le_iris.fit_transform(y_iris)

print("X shape:", X_iris.shape)
print("y length:", len(y_iris_encoded))
print("Classes:", le_iris.classes_)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode Cat
le_cat = LabelEncoder()
y_c3f_cat_enc = le_cat.fit_transform(y_c3f_cat)

# Encode Sub_Cat
le_sub = LabelEncoder()
y_c3f_sub_enc = le_sub.fit_transform(y_c3f_sub)

# Encode Label
le_label = LabelEncoder()
y_c3f_label_enc = le_label.fit_transform(y_c3f_label)

print("Cat classes:", le_cat.classes_)
print("SubCat classes:", len(le_sub.classes_))
print("Label classes:", len(le_label.classes_))


## computing complexity/meta-features for each dataset

In [ ]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

class DataComplexityAnalyzer:
    """
    Calculates data complexity measures (Ho & Basu + extensions).
    """

    def safe_cv(self, y, max_cv=3):
        """
        Choose number of CV folds based on minimum class size.
        If any class has <2 samples, return None (skip CV).
        """
        counts = np.bincount(y)
        min_class_size = np.min(counts)
        if min_class_size < 2:
            return None  # too few samples for CV
        return min(max_cv, min_class_size)

    def calculate_n1_borderline_points(self, X, y):
        if len(X) < 6: 
            return 0.0
        k = min(6, len(X))
        nn = NearestNeighbors(n_neighbors=k, metric='euclidean').fit(X)
        _, indices = nn.kneighbors(X)
        borderline = sum(len(np.unique(y[indices[i][1:]])) > 1 for i in range(len(X)))
        return borderline / len(X)

    def calculate_n2_intra_class_ratio(self, X, y):
        if len(X) < 4: 
            return 1.0
        nn = NearestNeighbors(n_neighbors=2, metric='euclidean').fit(X)
        dists, indices = nn.kneighbors(X)
        intra, inter = [], []
        for i in range(len(X)):
            ni, d = indices[i][1], dists[i][1]
            (intra if y[i] == y[ni] else inter).append(d)
        return np.mean(intra)/np.mean(inter) if intra and inter else 1.0

    def calculate_n3_error_rate(self, X, y):
        if len(X) < 3: 
            return 0.5
        errors = 0
        for i in range(len(X)):
            X_train, y_train = np.delete(X, i, 0), np.delete(y, i)
            nn = NearestNeighbors(n_neighbors=1).fit(X_train)
            pred = y_train[nn.kneighbors(X[i].reshape(1, -1))[1][0][0]]
            errors += (pred != y[i])
        return errors / len(X)

    def calculate_l2_linear_separability(self, X, y):
        if len(X) < 10: 
            return 0.5
        cv = self.safe_cv(y)
        if cv is None: 
            return 0.5
        clf = LogisticRegression(max_iter=200)
        return 1 - np.mean(cross_val_score(clf, X, y, cv=cv))

    def calculate_l3_nonlinear_separability(self, X, y):
        if len(X) < 10: 
            return 0.5
        cv = self.safe_cv(y)
        if cv is None: 
            return 0.5
        clf = DecisionTreeClassifier(max_depth=5)
        return 1 - np.mean(cross_val_score(clf, X, y, cv=cv))

    def calculate_f1_feature_efficiency(self, X, y):
        cv = self.safe_cv(y)
        if cv is None: 
            return 0.5
        max_eff = 0
        for j in range(X.shape[1]):
            clf = DecisionTreeClassifier(max_depth=1)
            scores = cross_val_score(clf, X[:, j].reshape(-1, 1), y, cv=cv)
            max_eff = max(max_eff, np.mean(scores))
        return 1 - max_eff

    def calculate_all_measures(self, X, y, target_name=""):
        if isinstance(X, pd.DataFrame): 
            X = X.values
        if isinstance(y, pd.Series): 
            y = y.values
        if not np.issubdtype(y.dtype, np.integer):
            y = LabelEncoder().fit_transform(y)

        measures = {
            "n1_borderline": self.calculate_n1_borderline_points(X, y),
            "n2_ratio": self.calculate_n2_intra_class_ratio(X, y),
            "n3_error": self.calculate_n3_error_rate(X, y),
            "l2_linear_sep": self.calculate_l2_linear_separability(X, y),
            "l3_nonlinear_sep": self.calculate_l3_nonlinear_separability(X, y),
            "f1_feature_eff": self.calculate_f1_feature_efficiency(X, y),
            "num_classes": len(np.unique(y)),
            "num_features": X.shape[1],
            "num_samples": X.shape[0]
        }
        return measures


In [ ]:
analyzer = DataComplexityAnalyzer()

iris_measures = analyzer.calculate_all_measures(X_iris.values, y_iris_encoded, target_name="iris")

print("\nIris Meta-Features:")
for k, v in iris_measures.items():
    print(f"{k}: {v}")


In [ ]:
print(X_iris.shape, len(y_iris_encoded))


In [ ]:
from sklearn.datasets import load_digits

# Load Digits
digits = load_digits(as_frame=True)
X_digits, y_digits = digits.data, digits.target

print("Digits shape:", X_digits.shape, "Classes:", len(np.unique(y_digits)))

# Run analyzer
digits_measures = analyzer.calculate_all_measures(X_digits.values, y_digits, target_name="digits")

print("\nDigits Meta-Features:")
for k, v in digits_measures.items():
    print(f"{k}: {v}")


In [ ]:
from sklearn.datasets import fetch_rcv1
from sklearn.decomposition import TruncatedSVD

# Load RCV1
rcv1 = fetch_rcv1()
X_rcv1, y_rcv1 = rcv1.data, rcv1.target

# Convert multi-label -> single-label (pick first label per sample)
y_rcv1_single = y_rcv1.argmax(axis=1).A1  

print("RCV1 full shape:", X_rcv1.shape, "Unique classes:", len(np.unique(y_rcv1_single)))

# Use only first 1000 samples
X_rcv1_small = X_rcv1[:1000]
y_rcv1_small = y_rcv1_single[:1000]

# Reduce dimensionality with SVD (from 47k → 200)
svd = TruncatedSVD(n_components=200, random_state=42)
X_rcv1_reduced = svd.fit_transform(X_rcv1_small)

print("Subset shape after reduction:", X_rcv1_reduced.shape)

# Run analyzer
rcv1_measures = analyzer.calculate_all_measures(X_rcv1_reduced, y_rcv1_small, target_name="rcv1_small")

print("\nRCV1 Meta-Features (subset of 1000, reduced to 200 features):")
for k, v in rcv1_measures.items():
    print(f"{k}: {v}")



In [ ]:
from sklearn.datasets import fetch_20newsgroups_vectorized
from sklearn.decomposition import TruncatedSVD

# Load vectorized 20 Newsgroups (TF-IDF)
news = fetch_20newsgroups_vectorized(subset="all")
X_20news, y_20news = news.data, news.target
target_names_20news = news.target_names

print("20 Newsgroups full shape:", X_20news.shape, "Unique classes:", len(np.unique(y_20news)))

# Take only first 500 samples
X_20news_small = X_20news[:500]
y_20news_small = y_20news[:500]

# Reduce dimensionality with TruncatedSVD (130k → 200 features)
svd = TruncatedSVD(n_components=200, random_state=42)
X_20news_reduced = svd.fit_transform(X_20news_small)

print("Subset shape after reduction:", X_20news_reduced.shape)

# Run analyzer
news_measures = analyzer.calculate_all_measures(X_20news_reduced, y_20news_small, target_name="20newsgroups_small")

print("\n20 Newsgroups Meta-Features (subset of 500, reduced to 200 features):")
for k, v in news_measures.items():
    print(f"{k}: {v}")



In [ ]:
import numpy as np
import pandas as pd

# Convert to float
X_c3f = X_c3f.astype(float)

# Replace inf / -inf with NaN
X_c3f = X_c3f.replace([np.inf, -np.inf], np.nan)

# Replace NaN with column mean (or median)
X_c3f = X_c3f.fillna(X_c3f.mean())

# Optional: clip extremely large values to a safe range
X_c3f = X_c3f.clip(lower=-1e6, upper=1e6)

print("Any NaN left?", X_c3f.isna().sum().sum())
print("Any inf left?", np.isinf(X_c3f.values).sum())


In [ ]:
# Monkey-patch: redefine calculate_n3_error_rate only for this analyzer instance
def fast_n3_error_rate(self, X, y):
    # Skip heavy leave-one-out when dataset is large
    if X.shape[0] > 500:
        return 0.5   # neutral default
    else:
        return DataComplexityAnalyzer.calculate_n3_error_rate(self, X, y)  # original method

# Apply patch
analyzer.calculate_n3_error_rate = fast_n3_error_rate.__get__(analyzer, DataComplexityAnalyzer)


In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress only LogisticRegression convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
# Use smaller sample size (e.g. 500) so other measures finish quickly
from sklearn.model_selection import train_test_split

X_c3f_small, _, y_cat_small, _ = train_test_split(
    X_c3f, y_c3f_cat_enc, train_size=500, stratify=y_c3f_cat_enc, random_state=42
)
_, _, y_sub_small, _ = train_test_split(
    X_c3f, y_c3f_sub_enc, train_size=500, stratify=y_c3f_sub_enc, random_state=42
)
_, _, y_label_small, _ = train_test_split(
    X_c3f, y_c3f_label_enc, train_size=500, stratify=y_c3f_label_enc, random_state=42
)

cat_measures   = analyzer.calculate_all_measures(X_c3f_small.values, y_cat_small,   target_name="c3f_cat")
sub_measures   = analyzer.calculate_all_measures(X_c3f_small.values, y_sub_small,   target_name="c3f_sub")
label_measures = analyzer.calculate_all_measures(X_c3f_small.values, y_label_small, target_name="c3f_label")


## Define action space

In [ ]:
action_space = [
    "xgb+lgb",
    "xgb",
    "lgb",
    "xgb+rf",
    "lgb+rf",
    "xgb+lgb+rf",
    "xgb+lgb+rf_meta",
    "xgb+lgb_meta",
    "xgb+rf_meta",
    "lgb+rf_meta",
    "logistic",
    "rf"
]

In [ ]:
action_map = {
    0: {"models": ["XGBoost", "LightGBM"], "meta": None},
    1: {"models": ["XGBoost"], "meta": None},
    2: {"models": ["LightGBM"], "meta": None},
    3: {"models": ["XGBoost", "RandomForest"], "meta": None},
    4: {"models": ["LightGBM", "RandomForest"], "meta": None},
    5: {"models": ["XGBoost", "LightGBM", "RandomForest"], "meta": None},
    6: {"models": ["XGBoost", "LightGBM", "RandomForest"], "meta": "LogisticRegression"},
    7: {"models": ["XGBoost", "LightGBM"], "meta": "LogisticRegression"},
    8: {"models": ["XGBoost", "RandomForest"], "meta": "LogisticRegression"},
    9: {"models": ["LightGBM", "RandomForest"], "meta": "LogisticRegression"},
    10: {"models": ["LogisticRegression"], "meta": None},
    11: {"models": ["RandomForest"], "meta": None}
}


In [ ]:
action_id = 3  # example choice
config = action_map[action_id]
print("Chosen models:", config["models"])
print("Meta-learner:", config["meta"])

In [ ]:
# Run analyzer
digits_measures = analyzer.calculate_all_measures(X_digits.values, y_digits, target_name="digits")

# Define which features are part of the RL state
state_keys = [
    "n1_borderline",
    "n2_ratio",
    "n3_error",
    "l2_linear_sep",
    "l3_nonlinear_sep",
    "f1_feature_eff",
    "num_classes",
    "num_features",
    "num_samples"
]

# Create state vector
state = np.array([digits_measures[k] for k in state_keys], dtype=float)
print("RL State vector (Digits):", state)


In [ ]:
def extract_state(measures, state_keys=None):
    if state_keys is None:
        state_keys = [
            "n1_borderline", "n2_ratio", "n3_error",
            "l2_linear_sep", "l3_nonlinear_sep", "f1_feature_eff",
            "num_classes", "num_features", "num_samples"
        ]
    return np.array([measures[k] for k in state_keys], dtype=float)


In [ ]:
state_digits = extract_state(digits_measures)
print(state_digits)


## Reward function

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier, VotingClassifier

def evaluate_action(X, y, action_id, action_map, test_size=0.2):
    # Split train/test
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)
    
    # Get config for this action
    config = action_map[action_id]
    models = config["models"]
    meta = config["meta"]
    
    # Build estimators
    estimators = []
    for m in models:
        if m == "RandomForest":
            estimators.append((m, RandomForestClassifier(random_state=42)))
        elif m == "XGBoost":
            estimators.append((m, XGBClassifier(use_label_encoder=False, eval_metric="mlogloss", random_state=42)))
        elif m == "LightGBM":
            estimators.append((m, LGBMClassifier(random_state=42)))
        elif m == "LogisticRegression":
            estimators.append((m, LogisticRegression(max_iter=500, random_state=42)))
    
    # Meta-learning (stacking)
    if meta:
        clf = StackingClassifier(
            estimators=estimators,
            final_estimator=LogisticRegression(max_iter=500, random_state=42),
            cv=3
        )
    else:
        # Simple voting
        if len(estimators) == 1:
            clf = estimators[0][1]   # just the single model
        else:
            clf = VotingClassifier(estimators=estimators, voting="soft")
    
    # Train & evaluate
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_val)
    acc = accuracy_score(y_val, y_pred)
    
    return acc


In [ ]:
# Example: evaluate Digits dataset with action 3 (xgb+rf)
reward = evaluate_action(X_digits.values, y_digits, action_id=3, action_map=action_map)
print("Reward (Validation Accuracy):", reward)


## Building Environment

In [ ]:
!pip install gymnasium

In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

class ModelSelectionEnv(gym.Env):
    def __init__(self, datasets, action_map, state_keys=None):
        super(ModelSelectionEnv, self).__init__()
        
        self.datasets = datasets
        self.action_map = action_map
        self.state_keys = state_keys or [
            "n1_borderline", "n2_ratio", "n3_error",
            "l2_linear_sep", "l3_nonlinear_sep", "f1_feature_eff",
            "num_classes", "num_features", "num_samples"
        ]
        
        self.action_space = spaces.Discrete(len(action_map))
        self.observation_space = spaces.Box(low=0, high=np.inf, 
                                            shape=(len(self.state_keys),), dtype=np.float32)
        self.current_dataset = None

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        import random
        self.current_dataset = random.choice(self.datasets)
        X, y, measures = self.current_dataset
        self.state = np.array([measures[k] for k in self.state_keys], dtype=float)
        return self.state, {}


    def step(self, action_id):
        X, y, measures = self.current_dataset
        reward = evaluate_action(X, y, action_id, self.action_map)
        done = True
        return self.state, reward, done, False, {}



In [ ]:
datasets = [
    (X_iris.values, y_iris_encoded, iris_measures),
    (X_digits.values, y_digits, digits_measures)
]

env = ModelSelectionEnv(datasets, action_map)

# Run one episode manually
state, info = env.reset()
print("Initial state:", state)

action = env.action_space.sample()
next_state, reward, done, trunc, _ = env.step(action)

print("Action taken:", action, "Reward:", reward)



In [ ]:
datasets = [
    (X_iris.values, y_iris_encoded, iris_measures),
    (X_digits.values, y_digits, digits_measures),
    (X_rcv1_reduced, y_rcv1_small, rcv1_measures),
    (X_20news_reduced, y_20news_small, news_measures),
    (X_c3f_small.values, y_cat_small, cat_measures),      # IoT Cat
    (X_c3f_small.values, y_sub_small, sub_measures),      # IoT Sub_Cat
    (X_c3f_small.values, y_label_small, label_measures)   # IoT Label
]


In [ ]:
env = ModelSelectionEnv(datasets, action_map)

# Run one random episode
state, info = env.reset()
print("Initial state:", state)

action = env.action_space.sample()
next_state, reward, done, trunc, _ = env.step(action)
print("Action taken:", action, "Reward:", reward)


## Choose RL Agent

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Original (subset + reduce)
X_rcv1_small = X_rcv1[:1000].toarray()
y_rcv1_small = y_rcv1_single[:1000]

# Encode labels to 0..N-1
le_rcv1 = LabelEncoder()
y_rcv1_encoded = le_rcv1.fit_transform(y_rcv1_small)

print("RCV1 Encoded classes:", np.unique(y_rcv1_encoded))


In [ ]:
datasets = [
    prepare_dataset(X_iris.values, y_iris_encoded, iris_measures),
    prepare_dataset(X_digits.values, y_digits, digits_measures),
    (X_rcv1_small, y_rcv1_encoded, rcv1_measures),     # <— use fixed encoded labels
    prepare_dataset(X_20news_reduced, y_20news_small, news_measures),
    prepare_dataset(X_c3f_small.values, y_cat_small, cat_measures),
    prepare_dataset(X_c3f_small.values, y_sub_small, sub_measures),
    prepare_dataset(X_c3f_small.values, y_label_small, label_measures)
]


In [ ]:
import numpy as np

class LinUCBAgent:
    def __init__(self, n_actions, n_features, alpha=1.0):
        self.n_actions = n_actions
        self.n_features = n_features
        self.alpha = alpha
        
        # For each action, track matrix A (dxd) and vector b (dx1)
        self.A = [np.identity(n_features) for _ in range(n_actions)]
        self.b = [np.zeros((n_features, 1)) for _ in range(n_actions)]
    
    def select_action(self, context):
        """
        Pick action using UCB score: p = θ^T x + α * sqrt(x^T A^-1 x)
        """
        context = context.reshape(-1, 1)
        p_values = []
        
        for a in range(self.n_actions):
            A_inv = np.linalg.inv(self.A[a])
            theta = A_inv @ self.b[a]
            mean = float(theta.T @ context)
            conf = self.alpha * np.sqrt(float(context.T @ A_inv @ context))
            p = mean + conf
            p_values.append(p)
        
        return np.argmax(p_values)
    
    def update(self, action, context, reward):
        """
        Update action-specific A and b with new observation
        """
        context = context.reshape(-1, 1)
        self.A[action] += context @ context.T
        self.b[action] += reward * context


## Training loop

In [ ]:
n_actions = len(action_map)
n_features = len(state)   # state is one meta-feature vector (len=9)

agent = LinUCBAgent(n_actions, n_features, alpha=1.0)

n_episodes = 50
for ep in range(n_episodes):
    state, info = env.reset()
    action = agent.select_action(state)
    _, reward, done, trunc, _ = env.step(action)
    agent.update(action, state, reward)
    print(f"Episode {ep+1}: Action={action}, Reward={reward:.3f}")


## Evaluating baselines

In [ ]:
from sklearn.preprocessing import LabelEncoder

# --- Fix RCV1 labels ---
le_rcv1 = LabelEncoder()
y_rcv1_encoded = le_rcv1.fit_transform(y_rcv1_small)
print("RCV1 encoded classes:", np.unique(y_rcv1_encoded))

# --- Fix 20News labels (just in case) ---
le_news = LabelEncoder()
y_20news_encoded = le_news.fit_transform(y_20news_small)
print("20News encoded classes:", np.unique(y_20news_encoded))


In [ ]:
datasets = [
    prepare_dataset(clean_features(X_iris.values), y_iris_encoded, iris_measures),
    prepare_dataset(clean_features(X_digits.values), y_digits, digits_measures),
    (clean_features(X_rcv1_small), y_rcv1_encoded, rcv1_measures),
    (clean_features(X_20news_reduced), y_20news_encoded, news_measures),
    prepare_dataset(clean_features(X_c3f_small.values), y_cat_small, cat_measures),
    prepare_dataset(clean_features(X_c3f_small.values), y_sub_small, sub_measures),
    prepare_dataset(clean_features(X_c3f_small.values), y_label_small, label_measures)
]

In [ ]:
env = ModelSelectionEnv(datasets, action_map)

In [ ]:
import random

# ----------------------------
# Random Policy Agent (baseline)
# ----------------------------
class RandomAgent:
    def __init__(self, n_actions):
        self.n_actions = n_actions
    
    def select_action(self, context):
        return random.randint(0, self.n_actions - 1)
    
    def update(self, action, context, reward):
        pass  # no learning for random agent

# ----------------------------
# Training Loop (Generic)
# ----------------------------
def run_agent(env, agent, n_episodes=100):
    rewards = []
    for ep in range(n_episodes):
        state, info = env.reset()
        action = agent.select_action(state)
        _, reward, done, trunc, _ = env.step(action)
        agent.update(action, state, reward)
        rewards.append(reward)
    return rewards


In [ ]:
n_actions = len(action_map)
n_features = len(state)   # length of one meta-feature vector

linucb_agent = LinUCBAgent(n_actions, n_features, alpha=1.0)
random_agent = RandomAgent(n_actions)

n_episodes = 100
linucb_rewards = run_agent(env, linucb_agent, n_episodes)
random_rewards = run_agent(env, random_agent, n_episodes)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.figure(figsize=(10,6))
plt.plot(np.cumsum(linucb_rewards) / (np.arange(len(linucb_rewards)) + 1),
         label="LinUCB (avg reward)", linewidth=2)
plt.plot(np.cumsum(random_rewards) / (np.arange(len(random_rewards)) + 1),
         label="Random Policy (avg reward)", linewidth=2, linestyle="--")

plt.xlabel("Episodes")
plt.ylabel("Average Reward (Validation Accuracy)")
plt.title("LinUCB vs Random Baseline")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Slimmed-down re-run for tuning alpha
alphas = [0.1, 0.5]
results = {}

for a in alphas:
    linucb_agent = LinUCBAgent(n_actions, n_features, alpha=a)  # reinit agent
    rewards = run_agent(env, linucb_agent, n_episodes=100)      # rerun training
    results[a] = rewards




In [ ]:
# Plot vs random baseline
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
for a, rewards in results.items():
    plt.plot(np.cumsum(rewards) / (np.arange(len(rewards)) + 1), label=f"LinUCB α={a}")

plt.plot(np.cumsum(random_rewards) / (np.arange(len(random_rewards)) + 1),
         label="Random Baseline", linestyle="--", color="black")

plt.xlabel("Episodes")
plt.ylabel("Average Reward")
plt.title("Tuning LinUCB α vs Random")
plt.legend()
plt.grid(True)
plt.show()

## Metrics report

In [ ]:
np.mean(linucb_rewards) #Average Reward (mean validation accuracy across episodes)

In [ ]:
np.max(linucb_rewards) #Best Reward (max accuracy achieved in any episode)

In [ ]:
np.std(linucb_rewards) #Stability (standard deviation of rewards)

In [ ]:
alphas = [0.1, 0.5, 1.0]
results = {}

for a in alphas:
    linucb_agent = LinUCBAgent(n_actions, n_features, alpha=a)
    rewards = run_agent(env, linucb_agent, n_episodes=100)
    results[a] = {
        "Avg Reward": np.mean(rewards),
        "Max Reward": np.max(rewards),
        "Std Dev": np.std(rewards)
    }

# Add Random baseline too
random_agent = RandomAgent(n_actions)
random_rewards = run_agent(env, random_agent, n_episodes=100)
results["Random"] = {
    "Avg Reward": np.mean(random_rewards),
    "Max Reward": np.max(random_rewards),
    "Std Dev": np.std(random_rewards)
}

# Print results
import pandas as pd
metrics_df = pd.DataFrame(results).T
print(metrics_df)


Avg Reward  Max Reward   Std Dev
0.1       0.668756    0.972222  0.277453
0.5       0.695589    0.972222  0.279794
1.0       0.633250    0.966667  0.296364
Random    0.684150    0.972222  0.270051

## Applying RL agaent to real-life(iot) dataset

In [ ]:
import pandas as pd
c3f=pd.read_csv(r"C:\Users\Kaveri\Downloads\CHASE datasets\c3f.csv")

In [ ]:
c3f['Label_num'] = c3f['Label'].astype('category').cat.codes

In [ ]:
from sklearn.model_selection import train_test_split

# First split: Train (70%) and temp (30%)
train_df, temp_df = train_test_split(
    c3f,
    test_size=0.30,
    stratify=c3f['Label_num'],
    random_state=42
)

# Second split: Validation (15%) and Test (15%) from temp
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.50,   # Half of 30% = 15%
    stratify=temp_df['Label_num'],
    random_state=42
)

print(f"Train shape: {train_df.shape}")
print(f"Validation shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")

In [ ]:
print("Train label distribution:\n", train_df['Label'].value_counts(normalize=True))
print("Validation label distribution:\n", val_df['Label'].value_counts(normalize=True))
print("Test label distribution:\n", test_df['Label'].value_counts(normalize=True))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif

# 2️⃣ Encode label for stratification
c3f['Label_num'] = c3f['Label'].astype('category').cat.codes

# 3️⃣ Split 70-15-15
train_df, temp_df = train_test_split(
    c3f,
    test_size=0.30,
    stratify=c3f['Label_num'],
    random_state=42
)

val_df, test_df = train_test_split(
    temp_df,
    test_size=0.50,
    stratify=temp_df['Label_num'],
    random_state=42
)

print(f"Train shape: {train_df.shape}")
print(f"Validation shape: {val_df.shape}")
print(f"Test shape: {test_df.shape}")

# 4️⃣ Cleaning function (no scaling yet)
def clean_features(df):
    features = df.drop(columns=['Cat', 'Sub_Cat', 'Label', 'Label_num'])
    features_numeric = features.select_dtypes(include=['int64', 'float64'])
    features_clean = features_numeric.copy()
    features_clean.replace([np.inf, -np.inf], np.nan, inplace=True)
    features_clean.fillna(features_clean.median(), inplace=True)
    return features_clean

X_train_clean = clean_features(train_df)
X_val_clean = clean_features(val_df)
X_test_clean = clean_features(test_df)

# 5️⃣ Scaling: fit on train, transform on val/test
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_clean)
X_val_scaled = scaler.transform(X_val_clean)
X_test_scaled = scaler.transform(X_test_clean)

# Convert back to DataFrames for feature names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_clean.columns, index=X_train_clean.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val_clean.columns, index=X_val_clean.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test_clean.columns, index=X_test_clean.index)

# 6️⃣ Mutual Information Feature Selection (on TRAIN ONLY, leakage-free)
feature_cols = X_train_scaled.columns.tolist()
targets = ['Cat', 'Sub_Cat', 'Label']
mi_results = {}

for target in targets:
    y_target = train_df[target]
    mi = mutual_info_classif(X_train_scaled, y_target, discrete_features='auto', random_state=42)
    mi_series = pd.Series(mi, index=feature_cols).sort_values(ascending=False)
    mi_results[target] = mi_series
    print(f"\n🔹 Top features for {target}:\n", mi_series.head(20))

# 7️⃣ Build final feature lists per level
cat_features = mi_results['Cat'].head(20).index.tolist()
subcat_features = mi_results['Sub_Cat'].head(20).index.tolist()
label_features = mi_results['Label'].head(20).index.tolist()

cross_level_features = list(set(cat_features) & set(subcat_features) & set(label_features))

print("\n🔹 Cross-level features:\n", cross_level_features)

cat_final_features = list(set(cat_features + cross_level_features))
subcat_final_features = list(set(subcat_features + cross_level_features))
label_final_features = list(set(label_features + cross_level_features))

print(f"\n✅ Final Cat features ({len(cat_final_features)}):\n", cat_final_features)
print(f"\n✅ Final Sub_Cat features ({len(subcat_final_features)}):\n", subcat_final_features)
print(f"\n✅ Final Label features ({len(label_final_features)}):\n", label_final_features)

In [ ]:
def check_determinism(df, target_col, previous_target_cols):
    """
    Check if target_col is perfectly determined by previous_target_cols.
    Returns True if deterministic, False otherwise.
    """
    if not previous_target_cols:
        return False

    # Build key from previous target(s)
    if len(previous_target_cols) == 1:
        combo_key = df[previous_target_cols[0]].astype(str)
    else:
        combo_key = df[previous_target_cols].astype(str).agg("_".join, axis=1)

    mapping = {}
    for combo, target in zip(combo_key, df[target_col]):
        if combo in mapping:
            if mapping[combo] != target:  # conflict found
                return False
        else:
            mapping[combo] = target

    return True


# 👉 Check determinism in IoT dataset
print("Is Sub_Cat deterministic from Cat?",
      check_determinism(c3f, "Sub_Cat", ["Cat"]))

print("Is Label deterministic from (Cat, Sub_Cat)?",
      check_determinism(c3f, "Label", ["Cat", "Sub_Cat"]))


In [ ]:
# 1️⃣ Extract X and y for each level
X_cat = X_train_scaled[cat_final_features]
y_cat = train_df['Cat']

X_sub = X_train_scaled[subcat_final_features]
y_sub = train_df['Sub_Cat']

X_label = X_train_scaled[label_final_features]
y_label = train_df['Label']

# 2️⃣ Compute complexity measures for each target level
cat_measures = analyzer.calculate_all_measures(X_cat, y_cat, target_name="IoT Cat")
subcat_measures = analyzer.calculate_all_measures(X_sub, y_sub, target_name="IoT SubCat")
label_measures = analyzer.calculate_all_measures(X_label, y_label, target_name="IoT Label")

# 3️⃣ Prepare IoT datasets for RL environment
iot_datasets = [
    prepare_dataset(X_cat, y_cat, cat_measures),
    prepare_dataset(X_sub, y_sub, subcat_measures),
    prepare_dataset(X_label, y_label, label_measures),
]

# 4️⃣ Build IoT-only environment
iot_env = ModelSelectionEnv(iot_datasets, action_map)


In [ ]:
# ----------------------------
# LinUCB Test Run on IoT Dataset
# ----------------------------

# Prepare IoT datasets (skip Label if deterministic)
iot_datasets = [
    prepare_dataset(X_cat, y_cat, cat_measures),
    prepare_dataset(X_sub, y_sub, subcat_measures),
]

if not check_determinism(c3f, "Label", ["Cat", "Sub_Cat"]):
    iot_datasets.append(prepare_dataset(X_label, y_label, label_measures))
else:
    print("⚡ Label is deterministic from (Cat, Sub_Cat) → skipping RL for Label.")

# Build IoT environment
iot_env = ModelSelectionEnv(iot_datasets, action_map)

# ----------------------------
# Run LinUCB on IoT
# ----------------------------
n_actions = len(action_map)

# FIX: reset once to get feature vector length
state, info = iot_env.reset()
n_features = len(state)

linucb_agent = LinUCBAgent(n_actions, n_features, alpha=0.5)  # you can tune alpha

n_episodes = 50
iot_rewards = []
iot_actions = []

for ep in range(n_episodes):
    state, info = iot_env.reset()
    action = linucb_agent.select_action(state)
    _, reward, done, trunc, _ = iot_env.step(action)
    linucb_agent.update(action, state, reward)
    iot_rewards.append(reward)
    iot_actions.append(action)
    print(f"Episode {ep+1}: Action={action_map[action]}, Reward={reward:.3f}")

# ----------------------------
# Summarize results
# ----------------------------
import pandas as pd

summary = pd.Series(iot_actions).map(action_map).value_counts()
print("\n📊 Model selection frequency on IoT dataset:")
print(summary)

print("\n✅ Average Reward:", np.mean(iot_rewards))
print("✅ Best Reward:", np.max(iot_rewards))


In [ ]:
## Level wise model reccommendations

In [ ]:
import pandas as pd
import numpy as np

# ----------------------------
# Helper: Run LinUCB on a given env
# ----------------------------
def run_linucb_on_env(env, n_episodes=50, alpha=0.5):
    state, info = env.reset()
    n_features = len(state)
    n_actions = len(action_map)

    agent = LinUCBAgent(n_actions, n_features, alpha=alpha)
    rewards, actions = [], []

    for ep in range(n_episodes):
        state, info = env.reset()
        action = agent.select_action(state)
        _, reward, done, trunc, _ = env.step(action)
        agent.update(action, state, reward)
        rewards.append(reward)
        actions.append(action)

    return rewards, actions


# ----------------------------
# 1️⃣ CAT ENVIRONMENT
# ----------------------------
iot_env_cat = ModelSelectionEnv(
    [prepare_dataset(X_cat, y_cat, cat_measures)], action_map
)
cat_rewards, cat_actions = run_linucb_on_env(iot_env_cat, n_episodes=50, alpha=0.5)

print("\n================ CAT RESULTS ================")
print(pd.Series(cat_actions).map(action_map).value_counts())
print("✅ Avg Reward (Val Acc):", np.mean(cat_rewards))
print("✅ Best Reward (Val Acc):", np.max(cat_rewards))


# ----------------------------
# 2️⃣ SUBCAT ENVIRONMENT
# ----------------------------
iot_env_sub = ModelSelectionEnv(
    [prepare_dataset(X_sub, y_sub, subcat_measures)], action_map
)
sub_rewards, sub_actions = run_linucb_on_env(iot_env_sub, n_episodes=50, alpha=0.5)

print("\n================ SUBCAT RESULTS ================")
print(pd.Series(sub_actions).map(action_map).value_counts())
print("✅ Avg Reward (Val Acc):", np.mean(sub_rewards))
print("✅ Best Reward (Val Acc):", np.max(sub_rewards))


# ----------------------------
# 3️⃣ LABEL CHECK
# ----------------------------
if check_determinism(c3f, "Label", ["Cat", "Sub_Cat"]):
    print("\n================ LABEL RESULTS ================")
    print("⚡ Label is deterministic from (Cat, Sub_Cat).")
    print("✅ No RL needed → Rule-based classification = 100% accuracy.")
else:
    iot_env_label = ModelSelectionEnv(
        [prepare_dataset(X_label, y_label, label_measures)], action_map
    )
    label_rewards, label_actions = run_linucb_on_env(iot_env_label, n_episodes=50, alpha=0.5)
    print("\n================ LABEL RESULTS ================")
    print(pd.Series(label_actions).map(action_map).value_counts())
    print("✅ Avg Reward (Val Acc):", np.mean(label_rewards))
    print("✅ Best Reward (Val Acc):", np.max(label_rewards))


================ CAT RESULTS ================
{'models': ['LightGBM', 'RandomForest'], 'meta': None}                               5
{'models': ['XGBoost', 'LightGBM', 'RandomForest'], 'meta': None}                    5
{'models': ['XGBoost', 'LightGBM', 'RandomForest'], 'meta': 'LogisticRegression'}    5
{'models': ['LightGBM', 'RandomForest'], 'meta': 'LogisticRegression'}               5
{'models': ['XGBoost', 'LightGBM'], 'meta': None}                                    4
{'models': ['XGBoost'], 'meta': None}                                                4
{'models': ['LightGBM'], 'meta': None}                                               4
{'models': ['XGBoost', 'RandomForest'], 'meta': None}                                4
{'models': ['XGBoost', 'LightGBM'], 'meta': 'LogisticRegression'}                    4
{'models': ['XGBoost', 'RandomForest'], 'meta': 'LogisticRegression'}                4
{'models': ['RandomForest'], 'meta': None}                                           4
{'models': ['LogisticRegression'], 'meta': None}                                     2
Name: count, dtype: int64
✅ Avg Reward (Val Acc): 0.9754124679760888
✅ Best Reward (Val Acc): 0.9826033914846896

In [ ]:
================ SUBCAT RESULTS ================
{'models': ['XGBoost', 'LightGBM', 'RandomForest'], 'meta': 'LogisticRegression'}    7
{'models': ['XGBoost', 'RandomForest'], 'meta': 'LogisticRegression'}                6
{'models': ['LightGBM', 'RandomForest'], 'meta': 'LogisticRegression'}               6
{'models': ['XGBoost', 'LightGBM'], 'meta': 'LogisticRegression'}                    5
{'models': ['XGBoost', 'LightGBM'], 'meta': None}                                    4
{'models': ['XGBoost'], 'meta': None}                                                4
{'models': ['LightGBM'], 'meta': None}                                               4
{'models': ['XGBoost', 'RandomForest'], 'meta': None}                                3
{'models': ['LightGBM', 'RandomForest'], 'meta': None}                               3
{'models': ['XGBoost', 'LightGBM', 'RandomForest'], 'meta': None}                    3
{'models': ['RandomForest'], 'meta': None}                                           3
{'models': ['LogisticRegression'], 'meta': None}                                     2
Name: count, dtype: int64
✅ Avg Reward (Val Acc): 0.8579397340490423
✅ Best Reward (Val Acc): 0.907844333292668

In [ ]:
# Save train/val/test splits to CSV for AutoML
train_df.to_csv("iot_train.csv", index=False)
val_df.to_csv("iot_val.csv", index=False)
test_df.to_csv("iot_test.csv", index=False)

print("✅ Saved train/val/test splits as CSVs.")
